In [8]:
import ssms 
import lanfactory
import os
import sys
import pickle
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [13]:
files_  = os.listdir('/users/afengler/data/proj_lan_pipeline/LAN_scripts/data/torch_models')
files_csv = ['/users/afengler/data/proj_lan_pipeline/LAN_scripts/data/torch_models/' + file_ for file_ in files_ if 'csv' in file_]

In [15]:
# TORCH
import os
import pandas as pd
import pickle
from copy import deepcopy 

#models = ['lca_no_bias_4', 'lca_no_bias_angle_4', 'race_no_bias_4', 'race_no_bias_angle_4']
#models = ['ddm_mic2_adj_no_bias', 'ddm_mic2_adj_weibull_no_bias', 'ddm_mic2_adj_angle_no_bias']

models = ['training_data_angle']

filters_ = models
path_ = '/users/afengler/data/proj_lan_pipeline/LAN_scripts/data/torch_models'
model_wise_pds = []

for filter_ in filters_:
    pds = {}
    models_tmp = deepcopy(models)
    models_tmp.remove(filter_)
    
    for file_ in os.listdir(path_):
        if filter_ in file_ and not (filter_ in models_tmp):
            if 'training_history' in file_:
                pd_tmp = pd.read_csv(path_ + '/' + file_)
                pd_tmp['model_id'] = file_[:file_.find('_')]
                pd_tmp['filename'] = file_
                pd_tmp['path'] = path_ + '/' + file_
                pd_tmp['model_type'] = filter_
                pds[file_[:file_.find('_')]] = pd_tmp


    for m_id in pds.keys():
        for file_ in os.listdir(path_):
            if m_id in file_ and 'network_config' in file_:
                network_config_tmp = pickle.load(open(path_ + '/' + file_, 'rb'))
                pds[m_id]['n_hidden_layers'] = len(network_config_tmp['layer_sizes']) - 1
                pds[m_id]['size_hidden_layers'] = network_config_tmp['layer_sizes'][0]
            #print(network_config_tmp)

    training_dat = pd.concat([pds[m_id] for m_id in pds.keys()]).reset_index(drop = True)

    best_models_pds = []
    for n_h_l in training_dat['n_hidden_layers'].unique():
        for s_h_l in training_dat['size_hidden_layers'].unique():
            training_dat_sub = training_dat.loc[(training_dat['n_hidden_layers'] == n_h_l) & \
                                                (training_dat['size_hidden_layers'] == s_h_l) & \
                                                (training_dat['epoch'] == training_dat['epoch'].max()), :]
            val_loss_min = training_dat_sub['val_loss'].min()
            best_models_pds.append(training_dat_sub.loc[training_dat_sub['val_loss'] == val_loss_min, :])
        
        
    best_models_dat = pd.concat(best_models_pds)
    model_wise_pds.append(best_models_dat)

training_dat = training_dat.loc[training_dat['epoch'] == training_dat['epoch'].max(), :]
full_dat = pd.concat(model_wise_pds).reset_index(drop = True).drop(['Unnamed: 0'], axis = 1)

In [17]:
full_dat

,epoch,val_loss,model_id,filename,path,model_type,n_hidden_layers,size_hidden_layers
0,19.0,0.002349,485bb72ea34911ec8d583cecef055870,485bb72ea34911ec8d583cecef055870_training_data...,/users/afengler/data/proj_lan_pipeline/LAN_scr...,training_data_angle,4,120
1,19.0,0.002339,24915962a33e11ecbe4ba0423f3e9be0,24915962a33e11ecbe4ba0423f3e9be0_training_data...,/users/afengler/data/proj_lan_pipeline/LAN_scr...,training_data_angle,4,100
2,19.0,0.002535,263c0ae6a33e11ec8c933cecef0559ac,263c0ae6a33e11ec8c933cecef0559ac_training_data...,/users/afengler/data/proj_lan_pipeline/LAN_scr...,training_data_angle,4,150
3,19.0,0.002612,cdf4d4c2a34611ecb44c3cecef05799c,cdf4d4c2a34611ecb44c3cecef05799c_training_data...,/users/afengler/data/proj_lan_pipeline/LAN_scr...,training_data_angle,3,120
4,19.0,0.002888,cfe591d6a34b11ecb0e3a0423f3e9a4a,cfe591d6a34b11ecb0e3a0423f3e9a4a_training_data...,/users/afengler/data/proj_lan_pipeline/LAN_scr...,training_data_angle,3,100
5,19.0,0.002565,676ab9f8a34911ec9aa43cecef0559ac,676ab9f8a34911ec9aa43cecef0559ac_training_data...,/users/afengler/data/proj_lan_pipeline/LAN_scr...,training_data_angle,3,150
6,19.0,0.002393,9adfaa96a34911ecbd183cecef055870,9adfaa96a34911ecbd183cecef055870_training_data...,/users/afengler/data/proj_lan_pipeline/LAN_scr...,training_data_angle,5,120
7,19.0,0.002259,248c94cca33e11ecb947ac1f6bfea5a4,248c94cca33e11ecb947ac1f6bfea5a4_training_data...,/users/afengler/data/proj_lan_pipeline/LAN_scr...,training_data_angle,5,100
8,19.0,0.002505,26424bc2a33e11ec90c83cecef0559ac,26424bc2a33e11ec90c83cecef0559ac_training_data...,/users/afengler/data/proj_lan_pipeline/LAN_scr...,training_data_angle,5,150


In [18]:
def get_summary(model = None,
                data = None,
                show_top_n = 10):
    
    val_loss = data['val_loss']
    data = data.sort_values('val_loss')
    print('val loss mean: ', data['val_loss'].mean())
    print('val loss min: ', data['val_loss'].min())
    print('val loss max: ', data['val_loss'].max())
    
    print('Top five network architectures')
    print(data.iloc[:show_top_n, :][['val_loss', 
                            'n_hidden_layers',
                            'size_hidden_layers']])
    
    print('Top five networks, paths')
    for i in range(show_top_n):
        print(data.iloc[i, :]['filename'])
        print(data.iloc[i, :]['path'])
    return data.iloc[:show_top_n, :]

In [19]:
from copy import deepcopy
get_summary(model = 'angle',
            data = deepcopy(training_dat))

val loss mean:  0.0026318251362276945
val loss min:  0.0022585825063288
val loss max:  0.0034407449420541
Top five network architectures
     val_loss  n_hidden_layers  size_hidden_layers
159  0.002259                5                 100
319  0.002329                5                 100
259  0.002339                4                 100
139  0.002349                4                 120
299  0.002393                5                 120
279  0.002505                5                 150
219  0.002535                4                 150
199  0.002551                5                 120
239  0.002558                4                 100
339  0.002565                3                 150
Top five networks, paths
248c94cca33e11ecb947ac1f6bfea5a4_training_data_angle_torch_training_history.csv
/users/afengler/data/proj_lan_pipeline/LAN_scripts/data/torch_models/248c94cca33e11ecb947ac1f6bfea5a4_training_data_angle_torch_training_history.csv
83e58a18a34e11ec95c2ac1f6bfea5a4_training_data_a

,Unnamed: 0,epoch,val_loss,model_id,filename,path,model_type,n_hidden_layers,size_hidden_layers
159,19,19.0,0.002259,248c94cca33e11ecb947ac1f6bfea5a4,248c94cca33e11ecb947ac1f6bfea5a4_training_data...,/users/afengler/data/proj_lan_pipeline/LAN_scr...,training_data_angle,5,100
319,19,19.0,0.002329,83e58a18a34e11ec95c2ac1f6bfea5a4,83e58a18a34e11ec95c2ac1f6bfea5a4_training_data...,/users/afengler/data/proj_lan_pipeline/LAN_scr...,training_data_angle,5,100
259,19,19.0,0.002339,24915962a33e11ecbe4ba0423f3e9be0,24915962a33e11ecbe4ba0423f3e9be0_training_data...,/users/afengler/data/proj_lan_pipeline/LAN_scr...,training_data_angle,4,100
139,19,19.0,0.002349,485bb72ea34911ec8d583cecef055870,485bb72ea34911ec8d583cecef055870_training_data...,/users/afengler/data/proj_lan_pipeline/LAN_scr...,training_data_angle,4,120
299,19,19.0,0.002393,9adfaa96a34911ecbd183cecef055870,9adfaa96a34911ecbd183cecef055870_training_data...,/users/afengler/data/proj_lan_pipeline/LAN_scr...,training_data_angle,5,120
279,19,19.0,0.002505,26424bc2a33e11ec90c83cecef0559ac,26424bc2a33e11ec90c83cecef0559ac_training_data...,/users/afengler/data/proj_lan_pipeline/LAN_scr...,training_data_angle,5,150
219,19,19.0,0.002535,263c0ae6a33e11ec8c933cecef0559ac,263c0ae6a33e11ec8c933cecef0559ac_training_data...,/users/afengler/data/proj_lan_pipeline/LAN_scr...,training_data_angle,4,150
199,19,19.0,0.002551,2462967ca33e11ecac2c3cecef055870,2462967ca33e11ecac2c3cecef055870_training_data...,/users/afengler/data/proj_lan_pipeline/LAN_scr...,training_data_angle,5,120
239,19,19.0,0.002558,8204f8eca34d11ec9a31a0423f3e9be0,8204f8eca34d11ec9a31a0423f3e9be0_training_data...,/users/afengler/data/proj_lan_pipeline/LAN_scr...,training_data_angle,4,100
339,19,19.0,0.002565,676ab9f8a34911ec9aa43cecef0559ac,676ab9f8a34911ec9aa43cecef0559ac_training_data...,/users/afengler/data/proj_lan_pipeline/LAN_scr...,training_data_angle,3,150


In [39]:
# Make dataframe that splits by unique n_hidden layers and unique size hidden layers
# --> Stores min val loss
# --> Stores filename

In [8]:
full_dat

,epoch,val_loss,model_id,filename,path,model_type,n_hidden_layers,size_hidden_layers
0,19.0,0.008604,72602cb2984011eca2c73cecef055870,72602cb2984011eca2c73cecef055870_ddm_par2_no_b...,/users/afengler/data/proj_lan_pipeline/LAN_scr...,ddm_par2_no_bias,4,120
1,19.0,0.008475,9b753046984711ecb2b5a0423f3e9a40,9b753046984711ecb2b5a0423f3e9a40_ddm_par2_no_b...,/users/afengler/data/proj_lan_pipeline/LAN_scr...,ddm_par2_no_bias,4,150
2,19.0,0.008581,d45b9534982f11eca4cd3cecef0571d2,d45b9534982f11eca4cd3cecef0571d2_ddm_par2_no_b...,/users/afengler/data/proj_lan_pipeline/LAN_scr...,ddm_par2_no_bias,4,100
3,19.0,0.008477,a930c68c984211ec90d83cecef056d34,a930c68c984211ec90d83cecef056d34_ddm_par2_no_b...,/users/afengler/data/proj_lan_pipeline/LAN_scr...,ddm_par2_no_bias,5,120
4,19.0,0.008562,f7c38c5a984511ecaea23cecef0571d2,f7c38c5a984511ecaea23cecef0571d2_ddm_par2_no_b...,/users/afengler/data/proj_lan_pipeline/LAN_scr...,ddm_par2_no_bias,5,150
5,19.0,0.008596,14abc0fe983b11ecb5223cecef0571d2,14abc0fe983b11ecb5223cecef0571d2_ddm_par2_no_b...,/users/afengler/data/proj_lan_pipeline/LAN_scr...,ddm_par2_no_bias,5,100
6,19.0,0.008573,c56e93ee984011ecb9c0a0423f3e9b5e,c56e93ee984011ecb9c0a0423f3e9b5e_ddm_par2_no_b...,/users/afengler/data/proj_lan_pipeline/LAN_scr...,ddm_par2_no_bias,3,120
7,19.0,0.008607,2cdbed24983711ec90e13cecef056d34,2cdbed24983711ec90e13cecef056d34_ddm_par2_no_b...,/users/afengler/data/proj_lan_pipeline/LAN_scr...,ddm_par2_no_bias,3,150
8,19.0,0.008713,d4692a78982f11eca78cac1f6bfea5a4,d4692a78982f11eca78cac1f6bfea5a4_ddm_par2_no_b...,/users/afengler/data/proj_lan_pipeline/LAN_scr...,ddm_par2_no_bias,3,100
